In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.io as sio
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define PINN with classifier
class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_classes):
        super(PINN, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes),
            nn.Softmax(dim=1)
        )
        self.regressor = nn.Sequential(
            nn.Linear(input_size + num_classes, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        class_probs = self.classifier(x)
        x_combined = torch.cat((x, class_probs), dim=1)
        return self.regressor(x_combined), class_probs

# Load and preprocess data from multiple files
def load_all_data(folder_path):
    X_list, y_list, labels_list = [], [], []
    max_length = 0

    # First pass to determine max length of Cp_matrix
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".mat"):
            file_path = os.path.join(folder_path, file_name)
            data = sio.loadmat(file_path)
            Cp_matrix = data.get('Wind_pressure_coefficients', np.zeros((1, 1)))
            max_length = max(max_length, Cp_matrix.shape[0] * Cp_matrix.shape[1])

    # Second pass to load data and ensure uniform Cp_matrix size
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".mat"):
            file_path = os.path.join(folder_path, file_name)
            data = sio.loadmat(file_path)

            wind_angle = float(data.get('Wind_azimuth', [[0]])[0][0])
            height = float(data.get('Building_height', [[1]])[0][0])
            depth = float(data.get('Building_depth', [[1]])[0][0])
            height_ratio = height / max(depth, 1e-6)  # Avoid division by zero
            eave_type = str(data.get('Eave_type', [['Unknown']])[0][0])  # Extract eave type

            Cp_matrix = data.get('Wind_pressure_coefficients', np.zeros((1, 1)))
            Cp_flat = Cp_matrix.flatten()
            if len(Cp_flat) < max_length:
                Cp_flat = np.pad(Cp_flat, (0, max_length - len(Cp_flat)), mode='constant')
            else:
                Cp_flat = Cp_flat[:max_length]

            X_list.append([wind_angle, height_ratio])
            y_list.append(Cp_flat)
            labels_list.append(eave_type)

    return np.array(X_list), np.array(y_list, dtype=float), np.array(labels_list, dtype=object)

# Directory containing multiple .mat files
data_folder = "../data/DATAAIO"
X, y, labels = load_all_data(data_folder)

# Encode categorical labels
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
labels_encoded = encoder.fit_transform(labels.reshape(-1, 1))
X_combined = np.hstack((X, labels_encoded))

# Standardize data
scaler_X = StandardScaler()
X_combined = scaler_X.fit_transform(X_combined)
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train, y_train = torch.tensor(X_train, dtype=torch.float32).to(device), torch.tensor(y_train, dtype=torch.float32).to(device)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32).to(device), torch.tensor(y_test, dtype=torch.float32).to(device)

# Model setup
input_size = X_train.shape[1]
hidden_size = 64
output_size = y_train.shape[1]
num_classes = labels_encoded.shape[1]
model = PINN(input_size, hidden_size, output_size, num_classes).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs, _ = model(X_train)
    loss = criterion(outputs, y_train)
    if torch.isnan(loss):  # Check for NaNs
        print("NaN detected in loss, skipping update")
        continue
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
    optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}')

# Save model
torch.save(model.state_dict(), "pinn_model.pth")
print("Model training complete and saved as pinn_model.pth")

# Validation
model.eval()
with torch.no_grad():
    y_pred, _ = model(X_test)
    y_pred = torch.nan_to_num(y_pred).cpu().numpy()  # Replace NaNs with zeros
    y_test = y_test.cpu().numpy()

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Validation Results:')
print(f'MAE: {mae:.4f}')
print(f'MSE: {mse:.4f}')
print(f'RMSE: {rmse:.4f}')
print(f'R²: {r2:.4f}')

Epoch [0/1000], Loss: 1.0677
Epoch [100/1000], Loss: 0.6583
Epoch [200/1000], Loss: 0.3658
Epoch [300/1000], Loss: 0.2514
Epoch [400/1000], Loss: 0.1884
Epoch [500/1000], Loss: 0.1467
Epoch [600/1000], Loss: 0.1159
Epoch [700/1000], Loss: 0.0902
Epoch [800/1000], Loss: 0.0690
Epoch [900/1000], Loss: 0.0526
Model training complete and saved as pinn_model.pth
Validation Results:
MAE: 0.5680
MSE: 0.6520
RMSE: 0.8075
R²: -610461775002861568.0000
